# エージェントの構築

LangChain は、[エージェント](/docs/concepts/agents)、つまり [LLM](/docs/concepts/chat_models) を推論エンジンとして用いて、実行するアクションとその実行に必要な入力を決定するシステムの作成をサポートしています。
アクションを実行した後、その結果を LLM にフィードバックすることで、さらにアクションが必要かどうか、あるいは終了してよいかどうかを判断できます。これは通常、[ツール呼び出し](/docs/concepts/tool_calling) によって実現されます。

このチュートリアルでは、検索エンジンと対話できるエージェントを構築します。このエージェントに質問したり、検索ツールを呼び出す様子を見たり、会話をしたりすることができます。

## エンドツーエンドエージェント

以下のコードスニペットは、LLM を使用して使用するツールを決定する、完全に機能するエージェントを表しています。汎用的な検索ツールを備えています。会話メモリを備えているため、マルチターンチャットボットとして使用できます。

ガイドの残りの部分では、個々のコンポーネントと各部分の役割について説明しますが、コードを取得して開始したい場合は、自由にこれを使用してください。

In [1]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = init_chat_model("anthropic:claude-3-5-sonnet-latest")
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [2]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I life in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob and I life in SF.
================================== Ai Message ==================================

Hello Bob! I notice you've introduced yourself and mentioned you live in SF (San Francisco), but you haven't asked a specific question or made a request that requires the use of any tools. Is there something specific you'd like to know about San Francisco or any other topic? I'd be happy to help you find information using the available search tools.


In [3]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================

[{'text': 'Let me search for current weather information in San Francisco.', 'type': 'text'}, {'id': 'toolu_011kSdheoJp8THURoLmeLtZo', 'input': {'query': 'current weather San Francisco CA'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_011kSdheoJp8THURoLmeLtZo)
 Call ID: toolu_011kSdheoJp8THURoLmeLtZo
  Args:
    query: current weather San Francisco CA
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather San Francisco CA", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, CA", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America',

## セットアップ

### Jupyter Notebook

このガイド（およびドキュメント内の他のほとんどのガイド）では、[Jupyter Notebook](https://jupyter.org/) を使用し、読者も同様に Jupyter Notebook を使用することを前提としています。Jupyter Notebook は、LLM システムの操作方法を学ぶのに最適なインタラクティブな環境です。なぜなら、予期しない出力や API のダウンなど、問題が発生することがよくあるからです。こうしたケースを観察することは、LLM を使った構築をより深く理解するための優れた方法です。

このチュートリアルやその他のチュートリアルは、Jupyter Notebook で実行するのが最も便利です。インストール方法については、[こちら](https://jupyter.org/install) をご覧ください。

### インストール

LangChain をインストールするには、次のコマンドを実行します。

In [ ]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

詳細については、[インストールガイド](/docs/how_to/installation)をご覧ください。

### LangSmith

LangChainで構築するアプリケーションの多くは、複数のステップでLLM呼び出しを複数回実行します。
これらのアプリケーションが複雑になるにつれて、チェーンまたはエージェント内で何が起こっているかを正確に検査することが重要になります。
これを行うには、[LangSmith](https://smith.langchain.com)を使用するのが最適です。

上記のリンクでサインアップしたら、環境変数を設定してトレースのログ記録を開始してください:

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

または、ノートブックの場合は次のように設定できます:

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

### Tavily

ツールとして[Tavily](/docs/integrations/tools/tavily_search)（検索エンジン）を使用します。
使用するには、APIキーを取得して設定する必要があります:

```bash
export TAVILY_API_KEY="..."
```

または、ノートブックの場合は次のように設定できます:

```python
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()
```

## ツールの定義

まず、使用するツールを作成する必要があります。メインツールとして選択するのは、検索エンジンの[Tavily](/docs/integrations/tools/tavily_search)です。専用の[langchain-tavily](https://pypi.org/project/langchain-tavily/) [統合パッケージ](/docs/concepts/architecture/#integration-packages)を使用することで、Tavily検索エンジンをLangChainのツールとして簡単に使用できます。


In [5]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco, CA', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1750168606, 'localtime': '2025-06-17 06:56'}, 'current': {'last_updated_epoch': 1750167900, 'last_updated': '2025-06-17 06:45', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 1, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/day/248.png', 'code': 1135}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 215, 'wind_dir': 'SW', 'pressure_mb': 1017.0, 'pressure_in': 30.02, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 0, 'feelslike_c': 11.3, 'feelslike_f': 52.4, 'windchill_c': 8.7, 'windchill_f': 47.7, 'heatindex_c': 9.8, 'heatindex_f': 49.7, 'dewpoint_c': 9.6, 'dewpoint_f':

:::tip

多くのアプリケーションでは、カスタムツールを定義する必要があるかもしれません。LangChainは、Python関数やその他の手段によるカスタムツールの作成をサポートしています。詳細については、[ツールの作成方法](/docs/how_to/custom_tools/)ガイドを参照してください。

:::

## 言語モデルの使用

次に、言語モデルを使ってツールを呼び出す方法を学びましょう。LangChainは様々な言語モデルをサポートしており、自由に使い分けることができます。以下からご希望の言語モデルを選択してください。

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4.1"}}} />


In [6]:
# | output: false
# | echo: false

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-latest")

メッセージのリストを渡すことで言語モデルを呼び出すことができます。デフォルトでは、レスポンスは `content` 文字列です。

In [7]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I help you today?'

このモデルでツール呼び出しができるようにすればどうなるかを見てみましょう。これを可能にするには、`.bind_tools` を使って言語モデルにこれらのツールに関する知識を与えます。

In [8]:
model_with_tools = model.bind_tools(tools)

これでモデルを呼び出すことができます。まずは通常のメッセージで呼び出して、どのような応答が返されるか確認してみましょう。`content` フィールドと `tool_calls` フィールドの両方を確認できます。

In [11]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! I'm here to help you. I have access to a powerful search tool that can help answer questions and find information about various topics. What would you like to know about?

Feel free to ask any question or request information, and I'll do my best to assist you using the available tools.

Tool calls: []


ここで、ツールが呼び出されると予想される入力でこれを呼び出してみましょう。

In [16]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: I'll help you search for information about the weather in San Francisco.

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'current weather San Francisco'}, 'id': 'toolu_015gdPn1jbB2Z21DmN2RAnti', 'type': 'tool_call'}]


テキストコンテンツは表示されていませんが、ツール呼び出しがあります。これはTavily Searchツールを呼び出すように指示しています。

これはまだツールを呼び出しているわけではなく、単に呼び出すように指示しているだけです。実際に呼び出すには、エージェントを作成する必要があります。

## エージェントを作成する

ツールとLLMを定義したので、エージェントを作成できます。エージェントの構築には[LangGraph](/docs/concepts/architecture/#langgraph)を使用します。
現在、エージェントの構築には高水準インターフェースを使用していますが、LangGraphの優れた点は、この高水準インターフェースが、エージェントのロジックを変更したい場合に備えて、低水準で高度に制御可能なAPIによって支えられていることです。

これで、LLMとツールを使ってエージェントを初期化できます。

`model_with_tools`ではなく`model`を渡していることに注意してください。これは、`create_react_agent`が内部的に`.bind_tools`を呼び出すためです。

In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## エージェントを実行する

これで、いくつかのクエリでエージェントを実行できるようになりました。現時点では、これらはすべて**ステートレス**なクエリです（以前のインタラクションは記憶されません）。エージェントはインタラクションの最後に**最終**状態を返します（これには入力も含まれます。出力のみを取得する方法については後ほど説明します）。

まず、ツールを呼び出す必要がない場合の応答を確認しましょう。

In [14]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! I'm here to help you with your questions using the available search tools. Please feel free to ask any question, and I'll do my best to find relevant and accurate information for you.


内部で何が起こっているかを正確に把握し（そしてツールを呼び出していないことを確認するために）、[LangSmith のトレース](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)を確認することができます。

では、ツールを呼び出すはずの例で試してみましょう。

In [15]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================

[{'text': "I'll help you search for weather information in San Francisco. Let me use the search engine to find current weather conditions.", 'type': 'text'}, {'id': 'toolu_01WWcXGnArosybujpKzdmARZ', 'input': {'query': 'current weather San Francisco SF'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01WWcXGnArosybujpKzdmARZ)
 Call ID: toolu_01WWcXGnArosybujpKzdmARZ
  Args:
    query: current weather San Francisco SF
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather San Francisco SF", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco, CA", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'r

[LangSmith トレース](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r)をチェックして、検索ツールが効果的に呼び出されていることを確認できます。

## メッセージのストリーミング

`.invoke` を使ってエージェントを呼び出し、最終的なレスポンスを取得する方法を見てきました。エージェントが複数のステップを実行する場合、処理に時間がかかることがあります。中間の進行状況を表示するには、発生したメッセージをストリーミングで返すことができます。

In [17]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================

[{'text': "I'll help you search for information about the weather in San Francisco.", 'type': 'text'}, {'id': 'toolu_01DCPnJES53Fcr7YWnZ47kDG', 'input': {'query': 'current weather San Francisco'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01DCPnJES53Fcr7YWnZ47kDG)
 Call ID: toolu_01DCPnJES53Fcr7YWnZ47kDG
  Args:
    query: current weather San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 

## トークンのストリーミング

メッセージのストリーミングに加えて、トークンのストリーミングも便利です。
これは、`stream_mode="messages"` を指定することで実現できます。


::: note

以下では `message.text()` を使用しますが、これには `langchain-core>=0.3.37` が必要です。

:::

In [18]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

I|'ll help you search for information| about the weather in San Francisco.|Base|d on the search results, here|'s the current weather in| San Francisco:
-| Temperature: 53.1°F (|11.7°C)
-| Condition: Foggy
- Wind:| 4.0 mph from| the Southwest
- Humidity|: 86%|
- Visibility: 9|.0 miles
- Pressure: |30.02 in|Hg

The weather| is characteristic of San Francisco, with| foggy conditions and mild temperatures|. The "feels like" temperature is slightly| lower at 52.4|°F (11.|3°C)| due to the wind chill effect|.|

## メモリへの追加

前述の通り、このエージェントはステートレスです。つまり、以前のインタラクションを記憶しません。メモリを確保するには、チェックポインタを渡す必要があります。チェックポインタを渡す際には、エージェントの呼び出し時に `thread_id` も渡す必要があります（これにより、エージェントはどのスレッド/会話から再開するかを認識できます）。

In [19]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [20]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [21]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hello Bob! I'm an AI assistant who can help you search for information using specialized search tools. Is there anything specific you'd like to know about or search for? I'm happy to help you find accurate and up-to-date information on various topics.


In [22]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob, as you introduced yourself earlier. I can remember information shared within our conversation without needing to search for it.


例 [LangSmith トレース](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

新しい会話を始めたい場合には、`thread_id`を変更するだけです。

In [23]:
# highlight-next-line
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I apologize, but I don't have access to any tools that would tell me your name. I can only assist you with searching for publicly available information using the tavily_search function. I don't have access to personal information about users. If you'd like to tell me your name, I'll be happy to address you by it.


## まとめ

これで終わりです！このクイックスタートでは、シンプルなエージェントの作成方法を説明しました。
さらに、中間ステップだけでなくトークンも含め、レスポンスをストリーミングで返す方法も説明しました。
さらに、エージェントと会話できるようにメモリも追加しました。
エージェントは複雑なトピックで、学ぶべきことがたくさんあります！

エージェントの詳細については、[LangGraph](/docs/concepts/architecture/#langgraph) のドキュメントをご覧ください。独自のコンセプト、チュートリアル、ハウツーガイドが掲載されています。